In [1]:
#競合を無視してPyCaretをインストール
!pip install pycaret --force-reinstall

ERROR: Could not find a version that satisfies the requirement pycaret (from versions: none)
ERROR: No matching distribution found for pycaret


In [2]:
import numpy as np
import pandas as pd

In [3]:
train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')

In [4]:
# object型の列名を取得するリストを作成
object_columns = []

# for文を使ってobject型の列を取得
for col in train.columns:
    if train[col].dtype == 'object':
        object_columns.append(col)

#取得した列にダミー変数を入れる
del object_columns[0]#先頭の'id'列を削除
cat_col = object_columns
X=pd.get_dummies(train,columns=cat_col)

In [5]:
X = X.drop(['id'],axis=1)

In [6]:
#K近傍法を実施
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
imputed_data = imputer.fit_transform(X)

In [7]:
columns = X.columns
index = X.index

imputed_df = pd.DataFrame(imputed_data, columns=columns, index=index)

In [8]:
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')

In [9]:
# 共通の列を抽出
common_columns = train.columns.intersection(test.columns)

# 訓練データとテストデータの共通列だけを抽出
train_common = train[common_columns]
test_common = test[common_columns]



# 訓練データ（およびテストデータ）のobject型の列名を取得するリストを作成
object_columns = []

# for文を使ってobject型の列を取得
for col in train_common.columns:
    if train_common[col].dtype == 'object':
        object_columns.append(col)



#訓練データとテストデータを合わせてダミー変数を入れる
data_combined = pd.concat([train_common, test_common], axis=0)
if object_columns:  # 空でない場合のみダミー変数化
    data_combined = pd.get_dummies(data_combined, columns=object_columns)
train_data = data_combined.iloc[:len(train_common), :]
test_data = data_combined.iloc[len(train_common):, :]

In [10]:
# 訓練データにsii列を追加
train_data['sii'] = imputed_df['sii']

/tmp/ipykernel_17/1926479172.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['sii'] = imputed_df['sii']


In [11]:
train_data['sii'] = train_data['sii'].astype(int)

/tmp/ipykernel_17/3493830447.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['sii'] = train_data['sii'].astype(int)


In [12]:
X = train_data
columns = X.columns
index = X.index

X = pd.DataFrame(X, columns=columns, index=index)

In [13]:
# PyCaretのセットアップ
from pycaret.classification import setup
clf = setup(data=X, target='sii', session_id=42, fold=5)  # foldでK分割数を指定

ModuleNotFoundError: No module named 'pycaret'

In [ ]:
# モデルの比較
from pycaret.classification import compare_models
best_model = compare_models()

In [ ]:
# 比較を実施し、上位3つのモデルを取得
models = compare_models(n_select=5)

# モデルのリストを確認
print(models)

In [ ]:
second_best_model = models[1]

In [ ]:
# 2位のモデルをチューニング
from pycaret.classification import tune_model
tuned_second_best_model = tune_model(second_best_model)

# チューニング後のモデルを確認
print(tuned_second_best_model)

In [ ]:
# モデルを評価
from pycaret.classification import evaluate_model
evaluate_model(tuned_second_best_model)

# 新しいデータに対する予測
from pycaret.classification import predict_model
predictions = predict_model(tuned_second_best_model, data=test_data)

In [ ]:
predictions.head(50)

In [ ]:
submission = pd.DataFrame({
    'id': test['id'],
    'sii': predictions['prediction_label']  # PyCaretは予測結果を"Label"列に格納
})

In [ ]:
submission.head(50)

In [ ]:
submission.to_csv("submission.csv", index=None)